# imports/setup

In [25]:
import datetime
import json
import pandas as pd
from collections import defaultdict
from typing import List, Dict, Any
import string
import jsonpickle
import numpy as np
import math
import io
from datamodel import Listing, ConversionObservation
from datamodel import TradingState, Listing, OrderDepth, Trade, Observation

# backtester

In [39]:
class Backtester:
    def __init__(self, trader, listings: Dict[str, Listing], position_limit: Dict[str, int], fair_marks, 
                 market_data: pd.DataFrame, trade_history: pd.DataFrame, file_name: str = None):
        self.trader = trader
        self.listings = listings
        self.market_data = market_data
        self.position_limit = position_limit
        self.fair_marks = fair_marks
        self.trade_history = trade_history.sort_values(by=['timestamp', 'symbol'])
        self.file_name = file_name

        self.observations = [Observation({}, {}) for _ in range(len(market_data))]

        self.current_position = {product: 0 for product in self.listings.keys()}
        self.pnl_history = []
        self.pnl = {product: 0 for product in self.listings.keys()}
        self.cash = {product: 0 for product in self.listings.keys()}
        self.trades = []
        self.sandbox_logs = []
        
    def run(self):
        traderData = ""
        
        timestamp_group_md = self.market_data.groupby('timestamp')
        timestamp_group_th = self.trade_history.groupby('timestamp')
        
        own_trades = defaultdict(list)
        market_trades = defaultdict(list)
        pnl_product = defaultdict(float)
        
        trade_history_dict = {}
        
        for timestamp, group in timestamp_group_th:
            trades = []
            for _, row in group.iterrows():
                symbol = row['symbol']
                price = row['price']
                quantity = row['quantity']
                buyer = row['buyer'] if pd.notnull(row['buyer']) else ""
                seller = row['seller'] if pd.notnull(row['seller']) else ""

                
                trade = Trade(symbol, int(price), int(quantity), buyer, seller, timestamp)
                
                trades.append(trade)
            trade_history_dict[timestamp] = trades
        
        
        for timestamp, group in timestamp_group_md:
            order_depths = self._construct_order_depths(group)
            order_depths_matching = self._construct_order_depths(group)
            order_depths_pnl = self._construct_order_depths(group)
            state = self._construct_trading_state(traderData, timestamp, self.listings, order_depths, 
                                 dict(own_trades), dict(market_trades), self.current_position, self.observations)
            orders, conversions, traderData = self.trader.run(state)
            products = group['product'].tolist()
            sandboxLog = ""
            trades_at_timestamp = trade_history_dict.get(timestamp, [])

            for product in products:
                new_trades = []
                for order in orders.get(product, []):
                    trades_done, sandboxLog = self._execute_order(timestamp, order, order_depths_matching, self.current_position, self.cash, trade_history_dict, sandboxLog)
                    new_trades.extend(trades_done)
                if len(new_trades) > 0:
                    own_trades[product] = new_trades
            self.sandbox_logs.append({"sandboxLog": sandboxLog, "lambdaLog": "", "timestamp": timestamp})

            trades_at_timestamp = trade_history_dict.get(timestamp, [])
            if trades_at_timestamp:
                for trade in trades_at_timestamp:
                    product = trade.symbol
                    market_trades[product].append(trade)
            else: 
                for product in products:
                    market_trades[product] = []

            
            for product in products:
                self._mark_pnl(self.cash, self.current_position, order_depths_pnl, self.pnl, product)
                self.pnl_history.append(self.pnl[product])
            self._add_trades(own_trades, market_trades)
        return self._log_trades(self.file_name)
    
    
    def _log_trades(self, filename: str = None):
        if filename is None:
            return 

        self.market_data['profit_and_loss'] = self.pnl_history

        output = ""
        output += "Sandbox logs:\n"
        for i in self.sandbox_logs:
            output += json.dumps(i, indent=2) + "\n"

        output += "\n\n\n\nActivities log:\n"
        market_data_csv = self.market_data.to_csv(index=False, sep=";")
        market_data_csv = market_data_csv.replace("\r\n", "\n")
        output += market_data_csv

        output += "\n\n\n\nTrade History:\n"
        output += json.dumps(self.trades, indent=2)

        with open(filename, 'w') as file:
            file.write(output)

            
    def _add_trades(self, own_trades: Dict[str, List[Trade]], market_trades: Dict[str, List[Trade]]):
        products = set(own_trades.keys()) | set(market_trades.keys())
        for product in products:
            self.trades.extend([self._trade_to_dict(trade) for trade in own_trades.get(product, [])])
        for product in products:
            self.trades.extend([self._trade_to_dict(trade) for trade in market_trades.get(product, [])])

    def _trade_to_dict(self, trade: Trade) -> dict[str, Any]:
        return {
            "timestamp": trade.timestamp,
            "buyer": trade.buyer,
            "seller": trade.seller,
            "symbol": trade.symbol,
            "currency": "SEASHELLS",
            "price": trade.price,
            "quantity": trade.quantity,
        }
        
    def _construct_trading_state(self, traderData, timestamp, listings, order_depths, 
                                 own_trades, market_trades, position, observations):
        state = TradingState(traderData, timestamp, listings, order_depths, 
                             own_trades, market_trades, position, observations)
        return state
    
        
    def _construct_order_depths(self, group):
        order_depths = {}
        for idx, row in group.iterrows():
            product = row['product']
            order_depth = OrderDepth()
            for i in range(1, 4):
                if f'bid_price_{i}' in row and f'bid_volume_{i}' in row:
                    bid_price = row[f'bid_price_{i}']
                    bid_volume = row[f'bid_volume_{i}']
                    if not pd.isna(bid_price) and not pd.isna(bid_volume):
                        order_depth.buy_orders[int(bid_price)] = int(bid_volume)
                if f'ask_price_{i}' in row and f'ask_volume_{i}' in row:
                    ask_price = row[f'ask_price_{i}']
                    ask_volume = row[f'ask_volume_{i}']
                    if not pd.isna(ask_price) and not pd.isna(ask_volume):
                        order_depth.sell_orders[int(ask_price)] = -int(ask_volume)
            order_depths[product] = order_depth
        return order_depths
    
        
        
    def _execute_buy_order(self, timestamp, order, order_depths, position, cash, trade_history_dict, sandboxLog):
        trades = []
        order_depth = order_depths[order.symbol]

        for price, volume in list(order_depth.sell_orders.items()):
            if price > order.price or order.quantity == 0:
                break

            trade_volume = min(abs(order.quantity), abs(volume))
            if abs(trade_volume + position[order.symbol]) <= int(self.position_limit[order.symbol]):
                trades.append(Trade(order.symbol, price, trade_volume, "SUBMISSION", "", timestamp))
                position[order.symbol] += trade_volume
                self.cash[order.symbol] -= price * trade_volume
                order_depth.sell_orders[price] += trade_volume
                order.quantity -= trade_volume
            else:
                sandboxLog += f"\nOrders for product {order.symbol} exceeded limit of {self.position_limit[order.symbol]} set"
            

            if order_depth.sell_orders[price] == 0:
                del order_depth.sell_orders[price]
        
        trades_at_timestamp = trade_history_dict.get(timestamp, [])
        new_trades_at_timestamp = []
        for trade in trades_at_timestamp:
            if trade.symbol == order.symbol:
                if trade.price < order.price:
                    trade_volume = min(abs(order.quantity), abs(trade.quantity))
                    trades.append(Trade(order.symbol, order.price, trade_volume, "SUBMISSION", "", timestamp))
                    order.quantity -= trade_volume
                    position[order.symbol] += trade_volume
                    self.cash[order.symbol] -= order.price * trade_volume
                    if trade_volume == abs(trade.quantity):
                        continue
                    else:
                        new_quantity = trade.quantity - trade_volume
                        new_trades_at_timestamp.append(Trade(order.symbol, order.price, new_quantity, "", "", timestamp))
                        continue
            new_trades_at_timestamp.append(trade)  

        if len(new_trades_at_timestamp) > 0:
            trade_history_dict[timestamp] = new_trades_at_timestamp

        return trades, sandboxLog
        
        
        
    def _execute_sell_order(self, timestamp, order, order_depths, position, cash, trade_history_dict, sandboxLog):
        trades = []
        order_depth = order_depths[order.symbol]
        
        for price, volume in sorted(order_depth.buy_orders.items(), reverse=True):
            if price < order.price or order.quantity == 0:
                break

            trade_volume = min(abs(order.quantity), abs(volume))
            if abs(position[order.symbol] - trade_volume) <= int(self.position_limit[order.symbol]):
                trades.append(Trade(order.symbol, price, trade_volume, "", "SUBMISSION", timestamp))
                position[order.symbol] -= trade_volume
                self.cash[order.symbol] += price * abs(trade_volume)
                order_depth.buy_orders[price] -= abs(trade_volume)
                order.quantity += trade_volume
            else:
                sandboxLog += f"\nOrders for product {order.symbol} exceeded limit of {self.position_limit[order.symbol]} set"

            if order_depth.buy_orders[price] == 0:
                del order_depth.buy_orders[price]

        trades_at_timestamp = trade_history_dict.get(timestamp, [])
        new_trades_at_timestamp = []
        for trade in trades_at_timestamp:
            if trade.symbol == order.symbol:
                if trade.price > order.price:
                    trade_volume = min(abs(order.quantity), abs(trade.quantity))
                    trades.append(Trade(order.symbol, order.price, trade_volume, "", "SUBMISSION", timestamp))
                    order.quantity += trade_volume
                    position[order.symbol] -= trade_volume
                    self.cash[order.symbol] += order.price * trade_volume
                    if trade_volume == abs(trade.quantity):
                        continue
                    else:
                        new_quantity = trade.quantity - trade_volume
                        new_trades_at_timestamp.append(Trade(order.symbol, order.price, new_quantity, "", "", timestamp))
                        continue
            new_trades_at_timestamp.append(trade)  

        if len(new_trades_at_timestamp) > 0:
            trade_history_dict[timestamp] = new_trades_at_timestamp
                
        return trades, sandboxLog
        
        
    def _execute_order(self, timestamp, order, order_depths, position, cash, trades_at_timestamp, sandboxLog):
        if order.quantity == 0:
            return [], sandboxLog
        
        order_depth = order_depths[order.symbol]
        if order.quantity > 0:
            return self._execute_buy_order(timestamp, order, order_depths, position, cash, trades_at_timestamp, sandboxLog)
        else:
            return self._execute_sell_order(timestamp, order, order_depths, position, cash, trades_at_timestamp, sandboxLog)

    def _mark_pnl(self, cash, position, order_depths, pnl, product):
        order_depth = order_depths[product]
        
        best_ask = min(order_depth.sell_orders.keys())
        best_bid = max(order_depth.buy_orders.keys())
        mid = (best_ask + best_bid)/2
        fair = mid
        if product in self.fair_marks:
            get_fair = self.fair_marks[product]
            fair = get_fair(order_depth)
        
        pnl[product] = cash[product] + fair * position[product]
        


# trader

In [70]:
from datamodel import OrderDepth, UserId, TradingState, Order, ConversionObservation
from typing import List, Dict, Any
import string
import jsonpickle
import numpy as np
import math


class Product:
    RAINFOREST_RESIN = "RAINFOREST_RESIN"
    KELP = "KELP"
    SQUID_INK = "SQUID_INK"
    CROISSANTS = "CROISSANTS"
    JAMS = "JAMS"
    DJEMBES = "DJEMBES"
    PICNIC_BASKET1 = "PICNIC_BASKET1"
    PICNIC_BASKET2 = "PICNIC_BASKET2"
    SYNTHETIC1 = "SYNTHETIC1"
    SYNTHETIC2 = "SYNTHETIC2"
    SPREAD1 = "SPREAD1"
    SPREAD2 = "SPREAD2"


PARAMS = {
    Product.RAINFOREST_RESIN: {
        "fair_value": 10000,
        "take_width": 1,
        "clear_width": 0,
        # for making
        "disregard_edge": 1,  # disregards orders for joining or pennying within this value from fair
        "join_edge": 2,  # joins orders within this edge
        "default_edge": 4,
        "soft_position_limit": 10,
    },
    Product.KELP: {
        "take_width": 1,
        "clear_width": -0.25,
        "prevent_adverse": True,
        "adverse_volume": 15,
        "reversion_beta": -0.229,
        "disregard_edge": 1,
        "join_edge": 0,
        "default_edge": 1,
    },
    Product.SQUID_INK: {
        "take_width": 1,
        "clear_width": -0.25,
        "prevent_adverse": True,
        "adverse_volume": 15,
        "reversion_beta": -0.229,
        "disregard_edge": 1,
        "join_edge": 0,
        "default_edge": 1,
    },
    Product.SPREAD1: {
        "default_spread_mean": 19837.280616341894,
        "default_spread_std": 62.29922153197848,
        "spread_sma_window": 1500,
        "spread_std_window": 45,
        "zscore_threshold": 7,
        "target_position": 39,
    },
    Product.SPREAD2: {
        "default_spread_mean": 45939.01799019319,
        "default_spread_std": 100.56850061727691,
        "spread_sma_window": 1500,
        "spread_std_window": 45,
        "zscore_threshold": 15,
        "target_position": 50,
    },
}

BASKET1_WEIGHTS = {
    Product.CROISSANTS: 6,
    Product.JAMS: 3,
    Product.DJEMBES: 1,
}

BASKET2_WEIGHTS = {
    Product.CROISSANTS: 4,
    Product.JAMS: 2,
}


class Trader:
    def __init__(self, params=None):
        if params is None:
            params = PARAMS
        self.params = params

        self.LIMIT = {
            Product.RAINFOREST_RESIN: 50,
            Product.KELP: 50,
            Product.SQUID_INK: 50,
            Product.PICNIC_BASKET1: 60,
            Product.PICNIC_BASKET2: 100,
            Product.CROISSANTS: 250,
            Product.JAMS: 350,
            Product.DJEMBES: 60,
        }

    # 以下函数保持原样不变……
    # Returns buy_order_volume, sell_order_volume
    def take_best_orders(
        self,
        product: str,
        fair_value: int,
        take_width: float,
        orders: List[Order],
        order_depth: OrderDepth,
        position: int,
        buy_order_volume: int,
        sell_order_volume: int,
        prevent_adverse: bool = False,
        adverse_volume: int = 0,
    ) -> (int, int):
        position_limit = self.LIMIT[product]
        if len(order_depth.sell_orders) != 0:
            best_ask = min(order_depth.sell_orders.keys())
            best_ask_amount = -1 * order_depth.sell_orders[best_ask]

            if best_ask <= fair_value - take_width:
                quantity = min(
                    best_ask_amount, position_limit - position
                )  # max amt to buy
                if quantity > 0:
                    orders.append(Order(product, best_ask, quantity))
                    buy_order_volume += quantity
                    order_depth.sell_orders[best_ask] += quantity
                    if order_depth.sell_orders[best_ask] == 0:
                        del order_depth.sell_orders[best_ask]

        if len(order_depth.buy_orders) != 0:
            best_bid = max(order_depth.buy_orders.keys())
            best_bid_amount = order_depth.buy_orders[best_bid]
            if best_bid >= fair_value + take_width:
                quantity = min(
                    best_bid_amount, position_limit + position
                )  # should be the max we can sell
                if quantity > 0:
                    orders.append(Order(product, best_bid, -1 * quantity))
                    sell_order_volume += quantity
                    order_depth.buy_orders[best_bid] -= quantity
                    if order_depth.buy_orders[best_bid] == 0:
                        del order_depth.buy_orders[best_bid]
        return buy_order_volume, sell_order_volume

    def take_best_orders_with_adverse(
        self,
        product: str,
        fair_value: int,
        take_width: float,
        orders: List[Order],
        order_depth: OrderDepth,
        position: int,
        buy_order_volume: int,
        sell_order_volume: int,
        adverse_volume: int,
    ) -> (int, int):
        position_limit = self.LIMIT[product]
        if len(order_depth.sell_orders) != 0:
            best_ask = min(order_depth.sell_orders.keys())
            best_ask_amount = -1 * order_depth.sell_orders[best_ask]
            if abs(best_ask_amount) <= adverse_volume:
                if best_ask <= fair_value - take_width:
                    quantity = min(
                        best_ask_amount, position_limit - position
                    )  # max amt to buy
                    if quantity > 0:
                        orders.append(Order(product, best_ask, quantity))
                        buy_order_volume += quantity
                        order_depth.sell_orders[best_ask] += quantity
                        if order_depth.sell_orders[best_ask] == 0:
                            del order_depth.sell_orders[best_ask]
        if len(order_depth.buy_orders) != 0:
            best_bid = max(order_depth.buy_orders.keys())
            best_bid_amount = order_depth.buy_orders[best_bid]
            if abs(best_bid_amount) <= adverse_volume:
                if best_bid >= fair_value + take_width:
                    quantity = min(
                        best_bid_amount, position_limit + position
                    )  # should be the max we can sell
                    if quantity > 0:
                        orders.append(Order(product, best_bid, -1 * quantity))
                        sell_order_volume += quantity
                        order_depth.buy_orders[best_bid] -= quantity
                        if order_depth.buy_orders[best_bid] == 0:
                            del order_depth.buy_orders[best_bid]
        return buy_order_volume, sell_order_volume

    def market_make(
        self,
        product: str,
        orders: List[Order],
        bid: int,
        ask: int,
        position: int,
        buy_order_volume: int,
        sell_order_volume: int,
    ) -> (int, int):
        buy_quantity = self.LIMIT[product] - (position + buy_order_volume)
        if buy_quantity > 0:
            orders.append(Order(product, round(bid), buy_quantity))  # Buy order
        sell_quantity = self.LIMIT[product] + (position - sell_order_volume)
        if sell_quantity > 0:
            orders.append(Order(product, round(ask), -sell_quantity))  # Sell order
        return buy_order_volume, sell_order_volume

    def clear_position_order(
        self,
        product: str,
        fair_value: float,
        width: int,
        orders: List[Order],
        order_depth: OrderDepth,
        position: int,
        buy_order_volume: int,
        sell_order_volume: int,
    ) -> List[Order]:
        position_after_take = position + buy_order_volume - sell_order_volume
        fair_for_bid = round(fair_value - width)
        fair_for_ask = round(fair_value + width)
        buy_quantity = self.LIMIT[product] - (position + buy_order_volume)
        sell_quantity = self.LIMIT[product] + (position - sell_order_volume)
        if position_after_take > 0:
            # Aggregate volume from all buy orders with price greater than fair_for_ask
            clear_quantity = sum(
                volume
                for price, volume in order_depth.buy_orders.items()
                if price >= fair_for_ask
            )
            clear_quantity = min(clear_quantity, position_after_take)
            sent_quantity = min(sell_quantity, clear_quantity)
            if sent_quantity > 0:
                orders.append(Order(product, fair_for_ask, -abs(sent_quantity)))
                sell_order_volume += abs(sent_quantity)
        if position_after_take < 0:
            # Aggregate volume from all sell orders with price lower than fair_for_bid
            clear_quantity = sum(
                abs(volume)
                for price, volume in order_depth.sell_orders.items()
                if price <= fair_for_bid
            )
            clear_quantity = min(clear_quantity, abs(position_after_take))
            sent_quantity = min(buy_quantity, clear_quantity)
            if sent_quantity > 0:
                orders.append(Order(product, fair_for_bid, abs(sent_quantity)))
                buy_order_volume += abs(sent_quantity)
        return buy_order_volume, sell_order_volume

    def kelp_fair_value(self, order_depth: OrderDepth, traderObject) -> float:
        if len(order_depth.sell_orders) != 0 and len(order_depth.buy_orders) != 0:
            best_ask = min(order_depth.sell_orders.keys())
            best_bid = max(order_depth.buy_orders.keys())
            filtered_ask = [
                price
                for price in order_depth.sell_orders.keys()
                if abs(order_depth.sell_orders[price])
                >= self.params[Product.KELP]["adverse_volume"]
            ]
            filtered_bid = [
                price
                for price in order_depth.buy_orders.keys()
                if abs(order_depth.buy_orders[price])
                >= self.params[Product.KELP]["adverse_volume"]
            ]
            mm_ask = min(filtered_ask) if len(filtered_ask) > 0 else None
            mm_bid = max(filtered_bid) if len(filtered_bid) > 0 else None
            if mm_ask == None or mm_bid == None:
                if traderObject.get("kelp_last_price", None) == None:
                    mmmid_price = (best_ask + best_bid) / 2
                else:
                    mmmid_price = traderObject["kelp_last_price"]
            else:
                mmmid_price = (mm_ask + mm_bid) / 2
            if traderObject.get("kelp_last_price", None) != None:
                last_price = traderObject["kelp_last_price"]
                last_returns = (mmmid_price - last_price) / last_price
                pred_returns = (
                    last_returns * self.params[Product.KELP]["reversion_beta"]
                )
                fair = mmmid_price + (mmmid_price * pred_returns)
            else:
                fair = mmmid_price
            traderObject["kelp_last_price"] = mmmid_price
            return fair
        return None

    def squid_ink_fair_value(self, order_depth: OrderDepth, traderObject) -> float:
        if len(order_depth.sell_orders) != 0 and len(order_depth.buy_orders) != 0:
            best_ask = min(order_depth.sell_orders.keys())
            best_bid = max(order_depth.buy_orders.keys())
            mmmid_price = (best_bid + best_ask) / 2
            if "squid_ink" not in traderObject:
                traderObject["squid_ink"] = {
                    "price_history": [],
                    "last_fair": mmmid_price,
                    "cycle_period": 100,  # Initial guess, will adapt
                    "phase": 0
                }
            squid_data = traderObject["squid_ink"]
            price_history = squid_data["price_history"]
            price_history.append(mmmid_price)
            if len(price_history) > 200:
                price_history.pop(0)
            avg_price = np.mean(price_history)
            std_dev = np.std(price_history)
            if len(price_history) >= 50:
                try:
                    fft_values = np.fft.fft(price_history - avg_price)
                    frequencies = np.fft.fftfreq(len(price_history))
                    dominant_freq = frequencies[np.argmax(np.abs(fft_values[1:])) + 1]
                    if dominant_freq != 0:
                        detected_period = int(abs(1 / dominant_freq))
                        squid_data["cycle_period"] = int(0.9 * squid_data["cycle_period"] + 0.1 * detected_period)
                except:
                    pass
                if len(price_history) >= 2:
                    if price_history[-1] > price_history[-2]:
                        squid_data["phase"] = (squid_data["phase"] + 5) % 360
                    else:
                        squid_data["phase"] = (squid_data["phase"] - 5) % 360
                cycle_period = max(10, min(200, squid_data["cycle_period"]))
                sine_factor = math.sin(2 * math.pi * squid_data["phase"] / 360)
                reversion_factor = (avg_price - mmmid_price) / (std_dev + 1e-6)
                sine_component = 0.5 * std_dev * sine_factor
                reversion_component = 0.3 * std_dev * np.tanh(reversion_factor)
                fair_value = avg_price + sine_component + reversion_component
                fair_value = max(best_bid, min(best_ask, fair_value))
                squid_data["last_fair"] = fair_value
                return fair_value
            if "squid_ink" in traderObject:
                return traderObject["squid_ink"]["last_fair"]
            return None
        
    def get_swmid(self, order_depth) -> float:
        best_bid = max(order_depth.buy_orders.keys())
        best_ask = min(order_depth.sell_orders.keys())
        best_bid_vol = abs(order_depth.buy_orders[best_bid])
        best_ask_vol = abs(order_depth.sell_orders[best_ask])
        return (best_bid * best_ask_vol + best_ask * best_bid_vol) / (
            best_bid_vol + best_ask_vol
        )
    def take_orders(
        self,
        product: str,
        order_depth: OrderDepth,
        fair_value: float,
        take_width: float,
        position: int,
        prevent_adverse: bool = False,
        adverse_volume: int = 0,
    ) -> (List[Order], int, int):
        orders: List[Order] = []
        buy_order_volume = 0
        sell_order_volume = 0
        if prevent_adverse:
            buy_order_volume, sell_order_volume = self.take_best_orders_with_adverse(
                product,
                fair_value,
                take_width,
                orders,
                order_depth,
                position,
                buy_order_volume,
                sell_order_volume,
                adverse_volume,
            )
        else:
            buy_order_volume, sell_order_volume = self.take_best_orders(
                product,
                fair_value,
                take_width,
                orders,
                order_depth,
                position,
                buy_order_volume,
                sell_order_volume,
            )
        return orders, buy_order_volume, sell_order_volume

    def clear_orders(
        self,
        product: str,
        order_depth: OrderDepth,
        fair_value: float,
        clear_width: int,
        position: int,
        buy_order_volume: int,
        sell_order_volume: int,
    ) -> (List[Order], int, int):
        orders: List[Order] = []
        buy_order_volume, sell_order_volume = self.clear_position_order(
            product,
            fair_value,
            clear_width,
            orders,
            order_depth,
            position,
            buy_order_volume,
            sell_order_volume,
        )
        return orders, buy_order_volume, sell_order_volume

    # ===== 以下为与spread交易相关的改动 =====

    # 修改后的 get_synthetic_basket_order_depth：根据 spread_type 计算 synthetic basket order depth  
    # 对于 spread1： synthetic basket = 6 CROISSANTS + 1 DJEMBES  
    # 对于 spread2： synthetic basket = 2 JAMS
    def get_synthetic_basket_order_depth(self, spread_type: str, order_depths: Dict[str, OrderDepth]) -> OrderDepth:
        synthetic_order_price = OrderDepth()
        if spread_type == "spread1":
            croissants_weight = 6
            djembes_weight = 1
            croissants_best_bid = max(order_depths[Product.CROISSANTS].buy_orders.keys()) if order_depths[Product.CROISSANTS].buy_orders else 0
            croissants_best_ask = min(order_depths[Product.CROISSANTS].sell_orders.keys()) if order_depths[Product.CROISSANTS].sell_orders else float("inf")
            djembes_best_bid = max(order_depths[Product.DJEMBES].buy_orders.keys()) if order_depths[Product.DJEMBES].buy_orders else 0
            djembes_best_ask = min(order_depths[Product.DJEMBES].sell_orders.keys()) if order_depths[Product.DJEMBES].sell_orders else float("inf")
            implied_bid = croissants_best_bid * croissants_weight + djembes_best_bid * djembes_weight
            implied_ask = croissants_best_ask * croissants_weight + djembes_best_ask * djembes_weight
            if implied_bid > 0:
                croissants_bid_volume = order_depths[Product.CROISSANTS].buy_orders[croissants_best_bid] // croissants_weight
                djembes_bid_volume = order_depths[Product.DJEMBES].buy_orders[djembes_best_bid] // djembes_weight
                implied_bid_volume = min(croissants_bid_volume, djembes_bid_volume)
                synthetic_order_price.buy_orders[implied_bid] = implied_bid_volume
            if implied_ask < float("inf"):
                croissants_ask_volume = -order_depths[Product.CROISSANTS].sell_orders[croissants_best_ask] // croissants_weight
                djembes_ask_volume = -order_depths[Product.DJEMBES].sell_orders[djembes_best_ask] // djembes_weight
                implied_ask_volume = min(croissants_ask_volume, djembes_ask_volume)
                synthetic_order_price.sell_orders[implied_ask] = -implied_ask_volume
        elif spread_type == "spread2":
            jams_weight = 2
            jams_best_bid = max(order_depths[Product.JAMS].buy_orders.keys()) if order_depths[Product.JAMS].buy_orders else 0
            jams_best_ask = min(order_depths[Product.JAMS].sell_orders.keys()) if order_depths[Product.JAMS].sell_orders else float("inf")
            implied_bid = jams_best_bid * jams_weight
            implied_ask = jams_best_ask * jams_weight
            if implied_bid > 0:
                jams_bid_volume = order_depths[Product.JAMS].buy_orders[jams_best_bid] // jams_weight
                synthetic_order_price.buy_orders[implied_bid] = jams_bid_volume
            if implied_ask < float("inf"):
                jams_ask_volume = -order_depths[Product.JAMS].sell_orders[jams_best_ask] // jams_weight
                synthetic_order_price.sell_orders[implied_ask] = -jams_ask_volume
        return synthetic_order_price

    # 修改后的 convert_synthetic_basket_orders：根据 spread_type 返回各 component 订单  
    # 对于 spread1：生成 CROISSANTS 与 DJEMBES 订单，数量按（6,1）放大；  
    # 对于 spread2：生成 JAMS 订单，数量按2倍放大。
    def convert_synthetic_basket_orders(self, spread_type: str, synthetic_orders: List[Order], order_depths: Dict[str, OrderDepth]) -> Dict[str, List[Order]]:
        if spread_type == "spread1":
            component_orders = {Product.CROISSANTS: [], Product.DJEMBES: []}
            synthetic_basket_order_depth = self.get_synthetic_basket_order_depth(spread_type, order_depths)
            best_bid = max(synthetic_basket_order_depth.buy_orders.keys()) if synthetic_basket_order_depth.buy_orders else 0
            best_ask = min(synthetic_basket_order_depth.sell_orders.keys()) if synthetic_basket_order_depth.sell_orders else float("inf")
            for order in synthetic_orders:
                price = order.price
                quantity = order.quantity
                if quantity > 0 and price >= best_ask:
                    croissants_price = min(order_depths[Product.CROISSANTS].sell_orders.keys())
                    djembes_price = min(order_depths[Product.DJEMBES].sell_orders.keys())
                    croissants_order = Order(Product.CROISSANTS, croissants_price, quantity * 6)
                    djembes_order = Order(Product.DJEMBES, djembes_price, quantity * 1)
                    component_orders[Product.CROISSANTS].append(croissants_order)
                    component_orders[Product.DJEMBES].append(djembes_order)
                elif quantity < 0 and price <= best_bid:
                    croissants_price = max(order_depths[Product.CROISSANTS].buy_orders.keys())
                    djembes_price = max(order_depths[Product.DJEMBES].buy_orders.keys())
                    croissants_order = Order(Product.CROISSANTS, croissants_price, quantity * 6)
                    djembes_order = Order(Product.DJEMBES, djembes_price, quantity * 1)
                    component_orders[Product.CROISSANTS].append(croissants_order)
                    component_orders[Product.DJEMBES].append(djembes_order)
            return component_orders
        elif spread_type == "spread2":
            component_orders = {Product.JAMS: []}
            synthetic_basket_order_depth = self.get_synthetic_basket_order_depth(spread_type, order_depths)
            best_bid = max(synthetic_basket_order_depth.buy_orders.keys()) if synthetic_basket_order_depth.buy_orders else 0
            best_ask = min(synthetic_basket_order_depth.sell_orders.keys()) if synthetic_basket_order_depth.sell_orders else float("inf")
            for order in synthetic_orders:
                price = order.price
                quantity = order.quantity
                if quantity > 0 and price >= best_ask:
                    jams_price = min(order_depths[Product.JAMS].sell_orders.keys())
                    jams_order = Order(Product.JAMS, jams_price, quantity * 2)
                    component_orders[Product.JAMS].append(jams_order)
                elif quantity < 0 and price <= best_bid:
                    jams_price = max(order_depths[Product.JAMS].buy_orders.keys())
                    jams_order = Order(Product.JAMS, jams_price, quantity * 2)
                    component_orders[Product.JAMS].append(jams_order)
            return component_orders

    # 修改后的 execute_spread_orders：增加 basket_product 与 synthetic_product 参数，根据 spread_type 执行转换
    def execute_spread_orders(
        self,
        target_position: int,
        basket_position: int,
        basket_product: str,
        synthetic_product: str,
        spread_type: str,
        order_depths: Dict[str, OrderDepth],
    ):
        if target_position == basket_position:
            return None
        target_quantity = abs(target_position - basket_position)
        basket_order_depth = order_depths[basket_product]
        synthetic_order_depth = self.get_synthetic_basket_order_depth(spread_type, order_depths)
        if target_position > basket_position:
            basket_ask_price = min(basket_order_depth.sell_orders.keys())
            basket_ask_volume = abs(basket_order_depth.sell_orders[basket_ask_price])
            synthetic_bid_price = max(synthetic_order_depth.buy_orders.keys())
            synthetic_bid_volume = abs(synthetic_order_depth.buy_orders[synthetic_bid_price])
            orderbook_volume = min(basket_ask_volume, synthetic_bid_volume)
            execute_volume = min(orderbook_volume, target_quantity)
            basket_orders = [Order(basket_product, basket_ask_price, execute_volume)]
            synthetic_orders = [Order(synthetic_product, synthetic_bid_price, -execute_volume)]
            aggregate_orders = self.convert_synthetic_basket_orders(spread_type, synthetic_orders, order_depths)
            aggregate_orders[basket_product] = basket_orders
            return aggregate_orders
        else:
            basket_bid_price = max(basket_order_depth.buy_orders.keys())
            basket_bid_volume = abs(basket_order_depth.buy_orders[basket_bid_price])
            synthetic_ask_price = min(synthetic_order_depth.sell_orders.keys())
            synthetic_ask_volume = abs(synthetic_order_depth.sell_orders[synthetic_ask_price])
            orderbook_volume = min(basket_bid_volume, synthetic_ask_volume)
            execute_volume = min(orderbook_volume, target_quantity)
            basket_orders = [Order(basket_product, basket_bid_price, -execute_volume)]
            synthetic_orders = [Order(synthetic_product, synthetic_ask_price, execute_volume)]
            aggregate_orders = self.convert_synthetic_basket_orders(spread_type, synthetic_orders, order_depths)
            aggregate_orders[basket_product] = basket_orders
            return aggregate_orders

    # 修改后的 spread_orders：根据传入的 spread_type、basket_product 以及对应的 traderObject 信息计算 zscore 并决定是否执行转换交易
    def spread_orders(self, spread_type: str, order_depths: Dict[str, OrderDepth], basket_product: str, basket_position: int, spread_data: Dict[str, Any]):
        if basket_product not in order_depths.keys():
            return None
        spread_param = self.params[Product.SPREAD1] if spread_type == "spread1" else self.params[Product.SPREAD2]
        basket_order_depth = order_depths[basket_product]
        synthetic_order_depth = self.get_synthetic_basket_order_depth(spread_type, order_depths)
        basket_swmid = self.get_swmid(basket_order_depth)
        synthetic_swmid = self.get_swmid(synthetic_order_depth)
        spread = basket_swmid - synthetic_swmid
        spread_data["spread_history"].append(spread)
        if len(spread_data["spread_history"]) < spread_param["spread_std_window"]:
            return None
        elif len(spread_data["spread_history"]) > spread_param["spread_std_window"]:
            spread_data["spread_history"].pop(0)
        spread_std = np.std(spread_data["spread_history"][-spread_param["spread_std_window"] :])
        spread_mean = spread_param["default_spread_mean"]
        zscore = (spread - spread_mean) / spread_std
        if zscore >= spread_param["zscore_threshold"]:
            if basket_position != -spread_param["target_position"]:
                return self.execute_spread_orders(
                    -spread_param["target_position"],
                    basket_position,
                    basket_product,
                    Product.SYNTHETIC1 if spread_type == "spread1" else Product.SYNTHETIC2,
                    spread_type,
                    order_depths,
                )
        if zscore <= -spread_param["zscore_threshold"]:
            if basket_position != spread_param["target_position"]:
                return self.execute_spread_orders(
                    spread_param["target_position"],
                    basket_position,
                    basket_product,
                    Product.SYNTHETIC1 if spread_type == "spread1" else Product.SYNTHETIC2,
                    spread_type,
                    order_depths,
                )
        return None
    
    def make_orders(
        self,
        product,
        order_depth: OrderDepth,
        fair_value: float,
        position: int,
        buy_order_volume: int,
        sell_order_volume: int,
        disregard_edge: float,  # disregard trades within this edge for pennying or joining
        join_edge: float,  # join trades within this edge
        default_edge: float,  # default edge to request if there are no levels to penny or join
        manage_position: bool = False,
        soft_position_limit: int = 0,
        # will penny all other levels with higher edge
    ):
        orders: List[Order] = []
        asks_above_fair = [
            price
            for price in order_depth.sell_orders.keys()
            if price > fair_value + disregard_edge
        ]
        bids_below_fair = [
            price
            for price in order_depth.buy_orders.keys()
            if price < fair_value - disregard_edge
        ]

        best_ask_above_fair = min(asks_above_fair) if len(asks_above_fair) > 0 else None
        best_bid_below_fair = max(bids_below_fair) if len(bids_below_fair) > 0 else None

        ask = round(fair_value + default_edge)
        if best_ask_above_fair != None:
            if abs(best_ask_above_fair - fair_value) <= join_edge:
                ask = best_ask_above_fair  # join
            else:
                ask = best_ask_above_fair - 1  # penny
        bid = round(fair_value - default_edge)
        if best_bid_below_fair != None:
            if abs(fair_value - best_bid_below_fair) <= join_edge:
                bid = best_bid_below_fair
            else:
                bid = best_bid_below_fair + 1

        if manage_position:
            if position > soft_position_limit:
                ask -= 1
            elif position < -1 * soft_position_limit:
                bid += 1

        buy_order_volume, sell_order_volume = self.market_make(
            product,
            orders,
            bid,
            ask,
            position,
            buy_order_volume,
            sell_order_volume,
        )

        return orders, buy_order_volume, sell_order_volume

    # ===== run 函数中的修改，仅针对spread部分 =====
    def run(self, state: TradingState):
        traderObject = {}
        if state.traderData is not None and state.traderData != "":
            traderObject = jsonpickle.decode(state.traderData)
        result = {}

        if Product.RAINFOREST_RESIN in self.params and Product.RAINFOREST_RESIN in state.order_depths:
            rainforest_resin_position = (
                state.position[Product.RAINFOREST_RESIN]
                if Product.RAINFOREST_RESIN in state.position
                else 0
            )
            rainforest_resin_take_orders, buy_order_volume, sell_order_volume = (
                self.take_orders(
                    Product.RAINFOREST_RESIN,
                    state.order_depths[Product.RAINFOREST_RESIN],
                    self.params[Product.RAINFOREST_RESIN]["fair_value"],
                    self.params[Product.RAINFOREST_RESIN]["take_width"],
                    rainforest_resin_position,
                )
            )
            rainforest_resin_clear_orders, buy_order_volume, sell_order_volume = (
                self.clear_orders(
                    Product.RAINFOREST_RESIN,
                    state.order_depths[Product.RAINFOREST_RESIN],
                    self.params[Product.RAINFOREST_RESIN]["fair_value"],
                    self.params[Product.RAINFOREST_RESIN]["clear_width"],
                    rainforest_resin_position,
                    buy_order_volume,
                    sell_order_volume,
                )
            )
            rainforest_resin_make_orders, _, _ = self.make_orders(
                Product.RAINFOREST_RESIN,
                state.order_depths[Product.RAINFOREST_RESIN],
                self.params[Product.RAINFOREST_RESIN]["fair_value"],
                rainforest_resin_position,
                buy_order_volume,
                sell_order_volume,
                self.params[Product.RAINFOREST_RESIN]["disregard_edge"],
                self.params[Product.RAINFOREST_RESIN]["join_edge"],
                self.params[Product.RAINFOREST_RESIN]["default_edge"],
                True,
                self.params[Product.RAINFOREST_RESIN]["soft_position_limit"],
            )
            result[Product.RAINFOREST_RESIN] = (
                rainforest_resin_take_orders + rainforest_resin_clear_orders + rainforest_resin_make_orders
            )

        if Product.KELP in self.params and Product.KELP in state.order_depths:
            kelp_position = (
                state.position[Product.KELP]
                if Product.KELP in state.position
                else 0
            )
            kelp_fair_value = self.kelp_fair_value(
                state.order_depths[Product.KELP], traderObject
            )
            kelp_take_orders, buy_order_volume, sell_order_volume = (
                self.take_orders(
                    Product.KELP,
                    state.order_depths[Product.KELP],
                    kelp_fair_value,
                    self.params[Product.KELP]["take_width"],
                    kelp_position,
                    self.params[Product.KELP]["prevent_adverse"],
                    self.params[Product.KELP]["adverse_volume"],
                )
            )
            kelp_clear_orders, buy_order_volume, sell_order_volume = (
                self.clear_orders(
                    Product.KELP,
                    state.order_depths[Product.KELP],
                    kelp_fair_value,
                    self.params[Product.KELP]["clear_width"],
                    kelp_position,
                    buy_order_volume,
                    sell_order_volume,
                )
            )
            kelp_make_orders, _, _ = self.make_orders(
                Product.KELP,
                state.order_depths[Product.KELP],
                kelp_fair_value,
                kelp_position,
                buy_order_volume,
                sell_order_volume,
                self.params[Product.KELP]["disregard_edge"],
                self.params[Product.KELP]["join_edge"],
                self.params[Product.KELP]["default_edge"],
            )
            result[Product.KELP] = (
                kelp_take_orders + kelp_clear_orders + kelp_make_orders
            )

        if Product.SQUID_INK in self.params and Product.SQUID_INK in state.order_depths:
            squid_ink_position = (
                state.position[Product.SQUID_INK]
                if Product.SQUID_INK in state.position
                else 0
            )
            squid_ink_fair_value = self.squid_ink_fair_value(
                state.order_depths[Product.SQUID_INK], traderObject
            )
            squid_ink_take_orders, buy_order_volume, sell_order_volume = (
                self.take_orders(
                    Product.SQUID_INK,
                    state.order_depths[Product.SQUID_INK],
                    squid_ink_fair_value,
                    self.params[Product.SQUID_INK]["take_width"],
                    squid_ink_position,
                    self.params[Product.SQUID_INK]["prevent_adverse"],
                    self.params[Product.SQUID_INK]["adverse_volume"],
                )
            )
            squid_ink_clear_orders, buy_order_volume, sell_order_volume = (
                self.clear_orders(
                    Product.SQUID_INK,
                    state.order_depths[Product.SQUID_INK],
                    squid_ink_fair_value,
                    self.params[Product.SQUID_INK]["clear_width"],
                    squid_ink_position,
                    buy_order_volume,
                    sell_order_volume,
                )
            )
            squid_ink_make_orders, _, _ = self.make_orders(
                Product.SQUID_INK,
                state.order_depths[Product.SQUID_INK],
                squid_ink_fair_value,
                squid_ink_position,
                buy_order_volume,
                sell_order_volume,
                self.params[Product.SQUID_INK]["disregard_edge"],
                self.params[Product.SQUID_INK]["join_edge"],
                self.params[Product.SQUID_INK]["default_edge"],
            )
            result[Product.SQUID_INK] = (
                squid_ink_take_orders + squid_ink_clear_orders + squid_ink_make_orders
            )

        # 对 spread 相关的数据分别保存到 traderObject 中
        if Product.SPREAD1 not in traderObject:
            traderObject[Product.SPREAD1] = {"spread_history": []}
        if Product.SPREAD2 not in traderObject:
            traderObject[Product.SPREAD2] = {"spread_history": []}

        basket1_position = state.position[Product.PICNIC_BASKET1] if Product.PICNIC_BASKET1 in state.position else 0
        basket2_position = state.position[Product.PICNIC_BASKET2] if Product.PICNIC_BASKET2 in state.position else 0

        spread1_orders = self.spread_orders("spread1", state.order_depths, Product.PICNIC_BASKET1, basket1_position, traderObject[Product.SPREAD1])
        spread2_orders = self.spread_orders("spread2", state.order_depths, Product.PICNIC_BASKET2, basket2_position, traderObject[Product.SPREAD2])
        if spread1_orders is not None:
            result[Product.PICNIC_BASKET1] = spread1_orders[Product.PICNIC_BASKET1]
            result[Product.CROISSANTS] = spread1_orders[Product.CROISSANTS]
            result[Product.DJEMBES] = spread1_orders[Product.DJEMBES]
        if spread2_orders is not None:
            result[Product.PICNIC_BASKET2] = spread2_orders[Product.PICNIC_BASKET2]
            result[Product.JAMS] = spread2_orders[Product.JAMS]

        conversions = 1
        traderData = jsonpickle.encode(traderObject)
        return result, conversions, traderData


# backtest run

In [61]:
def _process_data_(file):
    with open(file, 'r') as file:
        log_content = file.read()
    sections = log_content.split('Sandbox logs:')[1].split('Activities log:')
    sandbox_log =  sections[0].strip()
    activities_log = sections[1].split('Trade History:')[0]
    # sandbox_log_list = [json.loads(line) for line in sandbox_log.split('\n')]
    trade_history =  json.loads(sections[1].split('Trade History:')[1])
    # sandbox_log_df = pd.DataFrame(sandbox_log_list)
    market_data_df = pd.read_csv(io.StringIO(activities_log), sep=";", header=0)
    trade_history_df = pd.json_normalize(trade_history)
    return market_data_df, trade_history_df

### setup

In [62]:
def calculate_kelp_fair(order_depth):
    # assumes order_depth has orders in it 
    best_ask = min(order_depth.sell_orders.keys())
    best_bid = max(order_depth.buy_orders.keys())
    filtered_ask = [price for price in order_depth.sell_orders.keys() if abs(order_depth.sell_orders[price]) >= 15]
    filtered_bid = [price for price in order_depth.buy_orders.keys() if abs(order_depth.buy_orders[price]) >= 15]
    mm_ask = min(filtered_ask) if len(filtered_ask) > 0 else best_ask
    mm_bid = max(filtered_bid) if len(filtered_bid) > 0 else best_bid

    mmmid_price = (mm_ask + mm_bid) / 2
    return mmmid_price

def calculate_squid_ink_fair(order_depth):
    # assumes order_depth has orders in it 
    best_ask = min(order_depth.sell_orders.keys())
    best_bid = max(order_depth.buy_orders.keys())
    filtered_ask = [price for price in order_depth.sell_orders.keys() if abs(order_depth.sell_orders[price]) >= 15]
    filtered_bid = [price for price in order_depth.buy_orders.keys() if abs(order_depth.buy_orders[price]) >= 15]
    mm_ask = min(filtered_ask) if len(filtered_ask) > 0 else best_ask
    mm_bid = max(filtered_bid) if len(filtered_bid) > 0 else best_bid

    mmmid_price = (mm_ask + mm_bid) / 2
    return mmmid_price
    
def calculate_rainforest_resin_fair(order_depth):
    return 10000

In [63]:
fair_calculations = {
    "RAINFOREST_RESIN": calculate_rainforest_resin_fair,
    "KELP": calculate_kelp_fair,
    "SQUID_INK": calculate_squid_ink_fair
}

listings = {
    'RAINFOREST_RESIN': Listing(symbol='RAINFOREST_RESIN', product='RAINFOREST_RESIN', denomination='SEASHELLS'),
    'KELP': Listing(symbol='KELP', product='KELP', denomination='SEASHELLS'),
    'SQUID_INK': Listing(symbol='SQUID_INK', product='SQUID_INK', denomination='SEASHELLS'),
    'CROISSANTS': Listing(symbol='CROISSANTS', product='CROISSANTS', denomination='SEASHELLS'),
    'DJEMBES': Listing(symbol='DJEMBES', product='DJEMBES', denomination='SEASHELLS'),
    'JAMS': Listing(symbol='JAMS', product='JAMS', denomination='SEASHELLS'),
    'PICNIC_BASKET1': Listing(symbol='PICNIC_BASKET1', product='PICNIC_BASKET1', denomination='SEASHELLS'),
    'PICNIC_BASKET2': Listing(symbol='PICNIC_BASKET2', product='PICNIC_BASKET2', denomination='SEASHELLS')
}

position_limit = {
    'RAINFOREST_RESIN': 50,    # Rare item
    'KELP': 50,                # Rare item
    'SQUID_INK': 50,           # Rare item
    'CROISSANTS': 250,         # Common basket component
    'DJEMBES': 60,            # Common basket component
    'JAMS': 350,               # Common basket component
    'PICNIC_BASKET1': 60,      # Basket product
    'PICNIC_BASKET2': 100       # Basket product
}

### RUN FROM DATABOTTLE

In [64]:
from datamodel import Listing, Observation
import pandas as pd
import json
import io

In [71]:
# with fair prediction
day = 1
market_data = pd.read_csv(f"/Users/liumuhao/Desktop/imc-prosperity/round-2-island-data-bottle/prices_round_2_day_{day}.csv", sep=";", header=0)
trade_history = pd.read_csv(f"/Users/liumuhao/Desktop/imc-prosperity/round-2-island-data-bottle/trades_round_2_day_{day}.csv", sep=";", header=0)


trader = Trader()
backtester = Backtester(trader, listings, position_limit, fair_calculations, market_data, trade_history, "trade_history_sim.log")
backtester.run()
print(backtester.pnl)


{'RAINFOREST_RESIN': 33472, 'KELP': 4673.0, 'SQUID_INK': 114.0, 'CROISSANTS': 1377.0, 'DJEMBES': -175.5, 'JAMS': 2200.0, 'PICNIC_BASKET1': 1307.5, 'PICNIC_BASKET2': -7425.0}


In [59]:
for i in range(0, 3):
    # with fair prediction
    day = i
    market_data = pd.read_csv(f"./round-3-island-data-bottle/prices_round_3_day_{day}.csv", sep=";", header=0)
    trade_history = pd.read_csv(f"./round-3-island-data-bottle/trades_round_3_day_{day}_nn.csv", sep=";", header=0)

    trader = Trader()
    backtester = Backtester(trader, listings, position_limit, fair_calculations, market_data, trade_history, f"clean_data_logs/trade_history_day_{i}.log")
    backtester.run()
    print(backtester.pnl)

FileNotFoundError: [Errno 2] No such file or directory: './round-3-island-data-bottle/prices_round_3_day_0.csv'

`{'AMETHYSTS': 14554, 'STARFRUIT': 14144.5}`

In [ ]:
day = 0
market_data = pd.read_csv(f"./round-1-island-data-bottle/prices_round_1_day_{day}.csv", sep=";", header=0)
trade_history = pd.read_csv(f"./round-1-island-data-bottle/trades_round_1_day_{day}_nn.csv", sep=";", header=0)

trader = Trader()
backtester = Backtester(trader, listings, position_limit, market_data, trade_history, "trade_history_sim.log")
backtester.run()
print(backtester.pnl)

# backtest gridsearch

In [ ]:
import itertools

def generate_param_combinations(param_grid):
    param_names = param_grid.keys()
    param_values = param_grid.values()
    combinations = list(itertools.product(*param_values))
    return [dict(zip(param_names, combination)) for combination in combinations]

In [ ]:
import os
from tqdm import tqdm

def run_backtests(trader, listings, position_limit, fair_calcs, market_data, trade_history, backtest_dir, param_grid, symbol):
    if not os.path.exists(backtest_dir):
        os.makedirs(backtest_dir)

    param_combinations = generate_param_combinations(param_grid[symbol])

    results = []
    for params in tqdm(param_combinations, desc=f"Running backtests for {symbol}", unit="backtest"):
        trader.params = {symbol: params}
        backtester = Backtester(trader, listings, position_limit, fair_calcs, market_data, trade_history)
        backtester.run()

        param_str = "-".join([f"{key}={value}" for key, value in params.items()])
        log_filename = f"{backtest_dir}/{symbol}_{param_str}.log"
        backtester._log_trades(log_filename)

        results.append((params, backtester.pnl[symbol]))

    return results

### setup

In [ ]:
listings = {
    'AMETHYSTS': Listing(symbol='AMETHYSTS', product='AMETHYSTS', denomination='SEASHELLS'),
    'STARFRUIT': Listing(symbol='STARFRUIT', product='STARFRUIT', denomination='SEASHELLS')
}

position_limit = {
    'AMETHYSTS': 20,
    'STARFRUIT': 20
}

fair_calculations = {
    "AMETHYSTS": calculate_amethysts_fair,
    "STARFRUIT": calculate_starfruit_fair
}


In [ ]:
day = 0
market_data = pd.read_csv(f"./round-1-island-data-bottle/prices_round_1_day_{day}.csv", sep=";", header=0)
trade_history = pd.read_csv(f"./round-1-island-data-bottle/trades_round_1_day_{day}_nn.csv", sep=";", header=0)



### run

In [ ]:
backtest_dir = "backtestruns"

param_grid = {
    Product.AMETHYSTS: {
        "fair_value": [10000],
        "take_width": [1],
        "clear_width": [0.5],
        "volume_limit": [0],
        # for making
        "disregard_edge": [1],  # disregards orders for joining or pennying within this value from fair
        "join_edge": [2],# joins orders within this edge 
        "default_edge": [4]
    },
    Product.STARFRUIT: {
        "take_width": [1],
        "clear_width": [0, -0.25],
        "prevent_adverse": [True],
        "adverse_volume": [15],
        "reversion_beta": [-0.229],
        # for making
        "disregard_edge": [1],
        "join_edge": [3],
        "default_edge": [5],
    },
}



trader = Trader()

amethyst_results = run_backtests(trader, listings, position_limit, fair_calculations, market_data, trade_history, backtest_dir, param_grid, "AMETHYSTS")
print("AMETHYSTS results:")
for params, pnl in amethyst_results: 
    print(params)
    print(f"pnl: {pnl}")
    print("="*80)

starfruit_results = run_backtests(trader, listings, position_limit, fair_calculations, market_data, trade_history, backtest_dir, param_grid, "STARFRUIT")
print("STARFRUIT results:")
for params, pnl in starfruit_results: 
    print(params)
    print(f"pnl: {pnl}")
    print("="*80)

## analyze

In [ ]:

def analyze_log_files(backtest_dir):
    log_files = [f for f in os.listdir(backtest_dir) if f.endswith('.log')]
    
    results = []
    for log_file in log_files:
        file_path = os.path.join(backtest_dir, log_file)
        
        # Extract symbol and parameters from the file name
        file_name = os.path.splitext(log_file)[0]
        print(file_name)
        symbol, params_str = file_name.split('-', 1)
        params = dict(param.split('=') for param in params_str.split('-'))
        
        # Read the contents of the log file
        with open(file_path, 'r') as file:
            log_content = file.read()
        
        # Store the symbol, parameters, and log content in the results
        results.append({
            'symbol': symbol,
            'params': params,
            'log_content': log_content
        })
    
    return results

# Analyze the log files
log_analysis_results = analyze_log_files(backtest_dir)

# Print the results
for result in log_analysis_results:
    print(f"Symbol: {result['symbol']}")
    print(f"Parameters: {result['params']}")
#     print(f"Log Content:\n{result['log_content']}\n")

In [1]:
import datetime
import json
import pandas as pd
from collections import defaultdict
from typing import List, Dict, Any
import string
import jsonpickle
import numpy as np
import math
import io
from datamodel import Listing, ConversionObservation
from datamodel import TradingState, Listing, OrderDepth, Trade, Observation

In [2]:
class Product:
    AMETHYSTS = "AMETHYSTS"
    STARFRUIT = "STARFRUIT"
    ORCHIDS = "ORCHIDS"
    GIFT_BASKET = "GIFT_BASKET"
    CHOCOLATE = "CHOCOLATE"
    STRAWBERRIES = "STRAWBERRIES"
    ROSES = "ROSES"
    SYNTHETIC = "SYNTHETIC"
    SPREAD = "SPREAD"

def create_params(sma_window, std_window, zscore_threshold, target_position):
    return {
        Product.AMETHYSTS: {
            "fair_value": 10000,
            "take_width": 1,
            "clear_width": 0.5,
            "volume_limit": 0,
        },
        Product.STARFRUIT: {
            "take_width": 1,
            "clear_width": 0,
            "prevent_adverse": True,
            "adverse_volume": 15,
            "reversion_beta": -0.229,
            "starfruit_min_edge": 2,
        },
        Product.ORCHIDS: {
            "make_edge": 2,
            "make_min_edge": 1,
            "make_probability": 0.566,
            "init_make_edge": 2,
            "min_edge": 0.5,
            "volume_avg_timestamp": 5,
            "volume_bar": 75,
            "dec_edge_discount": 0.8,
            "step_size": 0.5,
        },
        Product.SPREAD:{
            "default_spread_mean": 379.50439988484237,
            "default_spread_std": 76.07966,
            "spread_sma_window": sma_window,
            "spread_std_window": std_window,
            "zscore_threshold": zscore_threshold,
            "target_position": target_position
        }
    }

In [4]:
from backtester import Backtester
from round3_v1_simple_strat import Trader
from tqdm import tqdm
import itertools

def _process_data_(file):
    with open(file, 'r') as file:
        log_content = file.read()
    sections = log_content.split('Sandbox logs:')[1].split('Activities log:')
    sandbox_log =  sections[0].strip()
    activities_log = sections[1].split('Trade History:')[0]
    # sandbox_log_list = [json.loads(line) for line in sandbox_log.split('\n')]
    trade_history =  json.loads(sections[1].split('Trade History:')[1])
    # sandbox_log_df = pd.DataFrame(sandbox_log_list)
    market_data_df = pd.read_csv(io.StringIO(activities_log), sep=";", header=0)
    trade_history_df = pd.json_normalize(trade_history)
    return market_data_df, trade_history_df

listings = {
    'AMETHYSTS': Listing(symbol='AMETHYSTS', product='AMETHYSTS', denomination='SEASHELLS'),
    'STARFRUIT': Listing(symbol='STARFRUIT', product='STARFRUIT', denomination='SEASHELLS'),
    'ORCHIDS': Listing(symbol='ORCHIDS', product='ORCHIDS', denomination='SEASHELLS'),
    'CHOCOLATE': Listing(symbol='CHOCOLATE', product='CHOCOLATE', denomination='SEASHELLS'),
    'STRAWBERRIES': Listing(symbol='STRAWBERRIES', product='STRAWBERRIES', denomination='SEASHELLS'),
    'ROSES': Listing(symbol='ROSES', product='ROSES', denomination='SEASHELLS'),
    'GIFT_BASKET': Listing(symbol='GIFT_BASKET', product='GIFT_BASKET', denomination='SEASHELLS'),
}

position_limit = {
    'AMETHYSTS': 20,
    'STARFRUIT': 20,
    'CHOCOLATE': 250,
    'STRAWBERRIES': 350,
    'ROSES': 60,
    'GIFT_BASKET': 60
}

fair_calculations = {}

market_data0, trade_history0 = _process_data_('./clean_log/trade_history_day_0.log')
market_data1, trade_history1 = _process_data_('./clean_log/trade_history_day_1.log')
market_data2, trade_history2 = _process_data_('./clean_log/trade_history_day_2.log')
market_data = market_data0
trade_history = trade_history0

sma_windows = [1500]
std_windows = [45]
zscore_thresholds = [9]
target_positions = [58]


for sma_window, std_window, zscore_threshold, target_position in tqdm(itertools.product(sma_windows, std_windows, zscore_thresholds, target_positions)):
    total_pnl = 0
    params = create_params(sma_window, std_window, zscore_threshold, target_position)
    trader = Trader(params=params)
    backtester = Backtester(trader, listings, position_limit, fair_calculations, market_data, trade_history, "simple_strat_backtest_no_clear_test_exceed.log")
    backtester.run()
    pnl = sum(float(pnl) for pnl in backtester.pnl.values())
    total_pnl += pnl
    print(f"sma_window: {sma_window}, std_window: {std_window}, zscore_threshold: {zscore_threshold}, target_position: {target_position}, pnl: {total_pnl}")
    print("="*80)



1it [00:47, 47.13s/it]

sma_window: 1500, std_window: 45, zscore_threshold: 9, target_position: 58, pnl: 46695.0
